In [1]:
import os
from os import listdir
from os.path import isfile, join

import pandas as pd
import time
import pickle
import shutil
import random
from  urllib.request import urlopen

In [2]:
# Paths
DATA_PATH = '../data/'
OUTPUT_PATH = '../data/raw/'
CREDS_PATH = '../collection/credentials/'

In [3]:
# Check jsonl in data folder
datafiles = [f for f in listdir(OUTPUT_PATH) if isfile(join(OUTPUT_PATH, f))]
print(datafiles)

['arxiv_GAN.jsonl', 'arxiv_GPT-3.jsonl', 'arxiv_fake news.jsonl', 'arxiv_disinformation.jsonl']


In [4]:
# Load df from jsonl
load_file = datafiles[1]

df = pd.read_json(OUTPUT_PATH + load_file, convert_dates=True, lines=True, orient='records')

In [5]:
# to process existing csv
csvfiles = [f for f in listdir(DATA_PATH) if isfile(join(DATA_PATH, f))]
print('Index, Filename')
print(list(zip([index for index, value in enumerate(csvfiles)], csvfiles)))

Index, Filename
[(0, 'deepfake_txt.csv'), (1, 'arxiv_disinformation.csv'), (2, 'arxiv_deepfake.csv'), (3, 'results.csv'), (4, 'reddit_machinelearning.csv'), (5, 'test_df.csv'), (6, 'mit_data.csv'), (7, 'importai_data.csv'), (8, 'cna_CNA_data.csv'), (9, 'synced_data.csv'), (10, 'ieee_data.csv')]


In [10]:
# Load df from csv
load_file = csvfiles[2]

CONVERTERS = {'tags': eval, 'arxiv_primary_category': eval,"published_parsed": eval, "links": eval}

df = pd.read_csv(DATA_PATH + load_file, converters=CONVERTERS)

In [11]:
df.tail(1)

,id,guidislink,link,updated,updated_parsed,published,published_parsed,title,title_detail,summary,...,arxiv_primary_category,tags,arxiv_affiliation,arxiv_journal_ref,arxiv_doi,cleaning,tokens,year,month_year,category
437,http://arxiv.org/abs/1806.02877v2,True,http://arxiv.org/abs/1806.02877v2,2018-06-11T19:28:49Z,"[2018, 6, 11, 19, 28, 49, 0, 162, 0]",2018-06-07T19:36:09Z,"[2018, 6, 7, 19, 36, 9, 3, 158, 0]",In Ictu Oculi: Exposing AI Generated Fake Face...,"{'type': 'text/plain', 'language': None, 'base...",The new developments in deep generative networ...,...,"{'term': 'cs.CV', 'scheme': 'http://arxiv.org/...","[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",NaN,NaN,NaN,the new developments in deep generative networ...,"['new', 'development', 'deep', 'generative', '...",2018,"[2018, 6]",cs.CV


In [13]:
# Checking the data we need
df.links[1][1]

{'title': 'pdf',
 'href': 'http://arxiv.org/pdf/2203.13964v1',
 'rel': 'related',
 'type': 'application/pdf'}

In [19]:
for index, row in df.iterrows():
      print(row.links[1]['title'])

pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf
pdf


KeyError: 'title'

In [20]:
# Creating name for directory creation

QUERY = load_file.split('_')[1].split('.')[0]
QUERY

'deepfake'

In [10]:
# for testing only
QUERY ='test'

In [11]:
# Make folder to hold PDFS
pdf_dir = DATA_PATH + QUERY + 'pdfs'

if not os.path.exists(pdf_dir): 
  os.makedirs(pdf_dir)


In [12]:
# Time out for our requests
timeout_secs = 10 

# List of PDFs we have
have = set(os.listdir(pdf_dir))

# Current and total downloads
numok = 0
numtot = 0

for index, row in df.iterrows():
  for item in row.links:
    if item['type'] == 'application/pdf':
      pdf_url = item['href']
      basename = pdf_url.split('/')[-1]
      fname = os.path.join(pdf_dir, basename)
      print(fname)

      try:
          if not basename in have:
            print('fetching %s into %s' % (pdf_url, fname))
            req = urlopen(pdf_url, None, timeout_secs)
            with open(fname, 'wb') as fp:
                shutil.copyfileobj(req, fp)
            time.sleep(0.05 + random.uniform(0,0.1))
          else:
            print('%s exists, skipping' % (fname, ))
          numok+=1
      except Exception as e:
          print('error downloading: ', pdf_url)
          print(e)
        
      print('%d/%d of %d downloaded ok.' % (numok, numtot, len(df)))
        
print('final number of papers downloaded okay: %d/%d' % (numok, len(df)))

../data/testpdfs/2204.02960v1
../data/testpdfs/2204.02960v1 exists, skipping
1/0 of 1206 downloaded ok.
../data/testpdfs/2204.02411v1
../data/testpdfs/2204.02411v1 exists, skipping
2/0 of 1206 downloaded ok.
../data/testpdfs/2204.02028v1
../data/testpdfs/2204.02028v1 exists, skipping
3/0 of 1206 downloaded ok.
../data/testpdfs/2204.00628v1
../data/testpdfs/2204.00628v1 exists, skipping
4/0 of 1206 downloaded ok.
../data/testpdfs/2204.01613v1
../data/testpdfs/2204.01613v1 exists, skipping
5/0 of 1206 downloaded ok.
../data/testpdfs/2204.01475v1
../data/testpdfs/2204.01475v1 exists, skipping
6/0 of 1206 downloaded ok.
../data/testpdfs/2204.01731v1
../data/testpdfs/2204.01731v1 exists, skipping
7/0 of 1206 downloaded ok.
../data/testpdfs/2204.01726v1
../data/testpdfs/2204.01726v1 exists, skipping
8/0 of 1206 downloaded ok.
../data/testpdfs/2204.01080v1
../data/testpdfs/2204.01080v1 exists, skipping
9/0 of 1206 downloaded ok.
../data/testpdfs/2204.01719v1
../data/testpdfs/2204.01719v1 exis